<a href="https://colab.research.google.com/github/damvas/politica_internacional/blob/main/cronologia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
# !pip install pdfplumber
import pdfplumber
import re
import pandas as pd
import os
import numpy as np

os.chdir('/content/drive/MyDrive/Python/Projetos/BD/Política Internacional')

def process_text(book_path: str) -> str:
  text = ""
  with pdfplumber.open(book_path) as pdf:
      for i in range(9, 253):
        page = pdf.pages[i]
        text += ' ' + (page.extract_text() or '')
  return text

def read_text_file(text_path: str) -> str:
  with open(text_path, 'r', encoding = 'utf-8') as file:
    text = file.read()
  text = text.replace(' 10\nCRONOLOGIA\n','')
  return text

def select_rows(text: str) -> list:
  rows = text.split('.\n')
  pattern = r'\.\s\d{2,3}\n'
  new_rows = []
  for row in rows:
    new_rows += re.split(pattern, row)
  return new_rows

def make_dataframe(new_rows: list) -> pd.DataFrame:
  df = pd.DataFrame(data = new_rows, columns = ['desc'])
  df['ano'] = df['desc'].apply(lambda x: re.findall(r'^(\d{4}) ', x)[0] if re.match(r'^\d{4} ', x) else None)
  df['desc'] = df['desc'].str.replace(r'^\d{4} ', '', regex=True)
  df['ano'] = df['ano'].fillna(method='ffill')
  df = df.iloc[:len(df)-1,:]
  df = df[['ano','desc']]
  df['desc'] = df['desc'].str.replace('\u2019',"'").str.replace('\n',' ')
  df['desc'] = df['desc'] + '.'
  return df

In [ ]:
# jogo de acertar o ano de tal acontecimento
# adicionar parâmetro de range de anos (mais difícil, 0, não pode ser nem um ano a mais ou a menos)

In [152]:
book_path = 'Cronologia das Relações Internacionais do Brasil.pdf'
text_path = 'cronologia_raw.txt'

text = read_text_file(text_path)
rows = select_rows(text)
df = make_dataframe(rows)

df.to_csv('cronologia.csv',index=False,sep=';',encoding='latin1')

In [169]:
num_questions = 1
year_range = 50
max_tries = 1

In [176]:
for i in range(num_questions):
  idx = np.random.randint(0, len(df) + 1)
  row = df.loc[idx]
  ano_correto = row['ano']
  desc = row['desc']

  ano_max = int(ano_correto) + year_range
  ano_min = int(ano_correto) - year_range

  correct = False
  tries = 0
  while not correct:
    guess = input(f'{desc}\nResposta: ')
    if guess.isdigit():
      if (int(guess) >= ano_min) & (int(guess) <= ano_max):
        print("\u2713")
        correct = True
      else:
        print("X")
        print(f'Gabarito: {ano_correto}')
        tries += 1
        if tries == max_tries:
          break
    else:
      print('Inserir ano')

São assinados nove acordos de cooperação e uma declaração conjunta.
Resposta: d
Inserir ano
São assinados nove acordos de cooperação e uma declaração conjunta.
Resposta: d
Inserir ano
São assinados nove acordos de cooperação e uma declaração conjunta.
Resposta: 2
X
Gabarito: 1980


True

In [136]:
def play():


,ano,desc
0,1492,"Buscando a rota para as Índias, o navegador ge..."
1,1493,"O papa Alexandre VI, em sua bula Inter Coetera..."
2,1494,"O Tratado de Tordesilhas, ou Capitulação da Pa..."
3,1498,O navegador português Vasco da Gama chega a Ca...
4,1498,"Duarte Pacheco Pereira, sob instruções secreta..."
...,...,...
3104,2016,Em reunião do Cone Sul sobre segurança nas fro...
3105,2016,Cai na Colômbia avião boliviano que transporta...
3106,2016,Promulgada emenda constitucional que limita os...
3107,2016,Resolução do Conselho de Segurança da ONU cond...
